In [4]:
import numpy as np
import pandas as pd
import zipfile
import gc
gc.enable()

In [5]:
train = "order_products__train"
products = 'products'
departments = 'departments'
aisles = 'aisles'
prior= "order_products__prior"
orders='orders'
path="/content/drive/MyDrive/Data_btp2/"
orders = zipfile.ZipFile(path+orders+".zip","r")
products = zipfile.ZipFile(path+products+".zip","r")
departments = zipfile.ZipFile(path+departments+".zip","r")
aisles = zipfile.ZipFile(path+aisles+".zip","r")
train = zipfile.ZipFile(path+train+".zip","r")
prior=zipfile.ZipFile(path+prior+".zip","r")

In [6]:
train = pd.read_csv(train.open('order_products__train.csv'))
prior =pd.read_csv(prior.open('order_products__prior.csv'))
products = pd.read_csv(products.open('products.csv'))
departments = pd.read_csv(departments.open('departments.csv'))
aisles = pd.read_csv(aisles.open('aisles.csv'))
orders = pd.read_csv(orders.open('orders.csv'))

In [7]:
orders.eval_set=orders.eval_set.replace({'prior':0, 'train':1,'test':2})

orders.days_since_prior_order=orders.days_since_prior_order.fillna(30)

In [8]:
prior_orders=pd.merge(orders,prior,on='order_id',how='inner')
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,0,1,2,8,30.0,196,1,0
1,2539329,1,0,1,2,8,30.0,14084,2,0
2,2539329,1,0,1,2,8,30.0,12427,3,0
3,2539329,1,0,1,2,8,30.0,26088,4,0
4,2539329,1,0,1,2,8,30.0,26405,5,0


In [9]:
del prior
gc.collect()

467

In [11]:
users=prior_orders.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()
users.head()

,user_id,u_num_of_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4
